In [ ]:
#Imports
import os #The OS module provides functions for interacting with the operating system. 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras

In [ ]:
#import cats_vs_dogs dataset from the modoule tensorflow_datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# split the data manually into 80% training, 10% testing, 10% validation
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
get_label_name = metadata.features['label'].int2str  
# creates a function object that we can use to get labels

# display 5 images from the dataset
for image, label in raw_train.take(5):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

In [ ]:
#data preprocessing: convert all images to the same size

IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  """
  returns an image that is reshaped to IMG_SIZE
  """
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [ ]:
#apply this function to all of the images using map()
train = raw_train.map(format_example)   #raw_* is how we imported the data
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [ ]:
#shuffle and batch the images
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [ ]:
#picking a pretrained model
#here we will use MobileNet V2

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
#freezing the base
base_model.trainable = False

In [ ]:
#adding our classifier

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#this flattens the layers from the pretrained model (1280 layers of 5x5) into a 1D tensor

In [ ]:
#add prediction layer (1 dense node because we are only predicting 2 classes)
prediction_layer = keras.layers.Dense(1)

In [ ]:
#combine the models and layers into a model
model = tf.keras.Sequential([
   base_model,
   global_average_layer,
   prediction_layer                          
])

In [ ]:
#training the model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# We can evaluate the model right now to see how it does before training it on our new images
initial_epochs = 3
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

In [ ]:
# Now we can train it on our images
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

acc = history.history['accuracy']
print(acc)

Epoch 1/3
582/582 [==============================] - 399s 685ms/step - loss: 0.2406 - accuracy: 0.8873 - val_loss: 0.0896 - val_accuracy: 0.9690
Epoch 2/3
582/582 [==============================] - 395s 679ms/step - loss: 0.0725 - accuracy: 0.9751 - val_loss: 0.0651 - val_accuracy: 0.9768
Epoch 3/3
582/582 [==============================] - 397s 681ms/step - loss: 0.0579 - accuracy: 0.9790 - val_loss: 0.0571 - val_accuracy: 0.9802
[0.8873186707496643, 0.9751209020614624, 0.9790435433387756]


In [ ]:
model.save("dogs_vs_cats.h5")  # we can save the model and reload it at anytime in the future
new_model = tf.keras.models.load_model('dogs_vs_cats.h5')